# Clone repo and install requirements

In [ ]:
%%bash
git clone https://github.com/nicolalandro/autovc.git
cd autovc
pip install univoc pyloudnorm

fatal: destination path 'autovc' already exists and is not an empty directory.


# Import requirements

In [ ]:
from IPython.display import Audio

# Read Audio

In [ ]:
# path1="autovc/wavs/p225/p225_003.wav"
path1="autovc/wavs/i300/galatea_01_barrili_f000001.wav"

In [ ]:
Audio(path1)

# Vocoder
Generate .wav audio file from generated spectrogram

In [ ]:
import librosa
import numpy as np
import pyloudnorm as pyln

In [ ]:
def melspectrogram(
    wav,
    sr=16000,
    hop_length=200,
    win_length=800,
    n_fft=2048,
    n_mels=128,
    fmin=50,
    preemph=0.97,
    top_db=80,
    ref_db=20,
):
    mel = librosa.feature.melspectrogram(
        librosa.effects.preemphasis(wav, coef=preemph),
        sr=sr,
        hop_length=hop_length,
        win_length=win_length,
        n_fft=n_fft,
        n_mels=n_mels,
        fmin=fmin,
        norm=1,
        power=1,
    )
    logmel = librosa.amplitude_to_db(mel, top_db=None) - ref_db
    logmel = np.maximum(logmel, -top_db)
    return logmel / top_db

In [ ]:
meter = pyln.Meter(16000)
wav, _ = librosa.load(path1, sr=16000)
loudness = meter.integrated_loudness(wav)
wav = pyln.normalize.loudness(wav, loudness, -24)
peak = np.abs(wav).max()
if peak >= 1:
  wav = wav / peak * 0.999
mel = melspectrogram(wav, n_mels=80)

In [ ]:
from univoc import Vocoder
import torch
import soundfile as sf

In [ ]:
vocoder = Vocoder.from_pretrained(
    "https://github.com/bshall/UniversalVocoding/releases/download/v0.2/univoc-ljspeech-7mtpaq.pt",
).cuda()

In [ ]:
torch_mel = torch.tensor(mel).unsqueeze(dim=0).transpose(1, 2)
torch_mel = torch_mel.cuda()
with torch.no_grad():
    wav2, sr = vocoder.generate(torch_mel)

In [ ]:
path2= "out.wav"
sf.write(path2, wav2, sr)

In [ ]:
Audio(path2)